# MongoDB Atlas + Fireworks AI RAG Example

In [ ]:
!pip install -q llama-index llama-index-vector-stores-mongodb llama-index-embeddings-fireworks==0.1.2 llama-index-llms-fireworks
!pip install -q pymongo datasets pandas

In [ ]:
# set up Fireworks.ai Key
import os
import getpass

fw_api_key = getpass.getpass("Fireworks API Key:")
os.environ["FIREWORKS_API_KEY"] = fw_api_key

In [ ]:
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/whatscooking.restaurants
dataset = load_dataset("AIatMongoDB/whatscooking.restaurants")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

/mnt/disks/data/llama_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,restaurant_id,attributes,cuisine,DogsAllowed,embedding,OutdoorSeating,borough,address,_id,name,menu,TakeOut,location,PriceRange,HappyHour,review_count,sponsored,stars
0,40366661,"{'Alcohol': ''none'', 'Ambience': '{'romantic'...",Tex-Mex,None,"[-0.14520384, 0.018315623, -0.018330636, -0.10...",True,Manhattan,"{'building': '627', 'coord': [-73.975980999999...",{'$oid': '6095a34a7c34416a90d3206b'},Baby Bo'S Burritos,None,True,"{'coordinates': [-73.97598099999999, 40.745132...",1.0,None,10,NaN,2.5
1,40367442,"{'Alcohol': ''beer_and_wine'', 'Ambience': '{'...",American,True,"[-0.11977468, -0.02157107, 0.0038846824, -0.09...",True,Staten Island,"{'building': '17', 'coord': [-74.1350211, 40.6...",{'$oid': '6095a34a7c34416a90d3209e'},Buddy'S Wonder Bar,"[Grilled cheese sandwich, Baked potato, Lasagn...",True,"{'coordinates': [-74.1350211, 40.6369042], 'ty...",2.0,None,62,NaN,3.5
2,40364610,"{'Alcohol': ''none'', 'Ambience': '{'touristy'...",American,None,"[-0.1004329, -0.014882699, -0.033005167, -0.09...",True,Staten Island,"{'building': '37', 'coord': [-74.138263, 40.54...",{'$oid': '6095a34a7c34416a90d31ff6'},Great Kills Yacht Club,"[Mozzarella sticks, Mushroom swiss burger, Spi...",True,"{'coordinates': [-74.138263, 40.546681], 'type...",1.0,None,72,NaN,4.0
3,40365288,"{'Alcohol': None, 'Ambience': '{'touristy': Fa...",American,None,"[-0.11735515, -0.0397448, -0.0072645755, -0.09...",True,Manhattan,"{'building': '842', 'coord': [-73.970637000000...",{'$oid': '6095a34a7c34416a90d32017'},Keats Restaurant,"[French fries, Chicken pot pie, Mac & cheese, ...",True,"{'coordinates': [-73.97063700000001, 40.751495...",2.0,True,149,NaN,4.0
4,40363151,"{'Alcohol': None, 'Ambience': None, 'BYOB': No...",Bakery,None,"[-0.096541286, -0.009661355, 0.04402167, -0.12...",True,Manhattan,"{'building': '120', 'coord': [-73.9998042, 40....",{'$oid': '6095a34a7c34416a90d31fbd'},Olive'S,"[doughnuts, chocolate chip cookies, chocolate ...",True,"{'coordinates': [-73.9998042, 40.7251256], 'ty...",1.0,None,7,NaN,5.0


In [ ]:
from llama_index.core.settings import Settings
from llama_index.llms.fireworks import Fireworks
from llama_index.embeddings.fireworks import FireworksEmbedding

embed_model = FireworksEmbedding(
    embed_batch_size=512,
    model_name="nomic-ai/nomic-embed-text-v1.5",
    api_key=fw_api_key,
)
llm = Fireworks(
    temperature=0,
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    api_key=fw_api_key,
)

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
import json
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

# Convert the DataFrame to a JSON string representation
documents_json = dataset_df.to_json(orient="records")
# Load the JSON string into a Python list of dictionaries
documents_list = json.loads(documents_json)

llama_documents = []

for document in documents_list:
    # Value for metadata must be one of (str, int, float, None)
    document["name"] = json.dumps(document["name"])
    document["cuisine"] = json.dumps(document["cuisine"])
    document["attributes"] = json.dumps(document["attributes"])
    document["menu"] = json.dumps(document["menu"])
    document["borough"] = json.dumps(document["borough"])
    document["address"] = json.dumps(document["address"])
    document["PriceRange"] = json.dumps(document["PriceRange"])
    document["HappyHour"] = json.dumps(document["HappyHour"])
    document["review_count"] = json.dumps(document["review_count"])
    document["TakeOut"] = json.dumps(document["TakeOut"])
    # these two fields are not relevant to the question we want to answer,
    # so I will skip it for now
    del document["embedding"]
    del document["location"]

    # Create a Document object with the text and excluded metadata for llm and embedding models
    llama_document = Document(
        text=json.dumps(document),
        metadata=document,
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )

    llama_documents.append(llama_document)

# Observing an example of what the LLM and Embedding model receive as input
print(
    "\nThe LLM sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.LLM),
)
print(
    "\nThe Embedding model sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED),
)


The LLM sees this: 
 Metadata: restaurant_id=>40366661
attributes=>{"Alcohol": "'none'", "Ambience": "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}", "BYOB": null, "BestNights": null, "BikeParking": null, "BusinessAcceptsBitcoin": null, "BusinessAcceptsCreditCards": null, "BusinessParking": "None", "Caters": "True", "DriveThru": null, "GoodForDancing": null, "GoodForKids": "True", "GoodForMeal": null, "HasTV": "True", "Music": null, "NoiseLevel": "'average'", "RestaurantsAttire": "'casual'", "RestaurantsDelivery": "True", "RestaurantsGoodForGroups": "True", "RestaurantsReservations": "True", "RestaurantsTableService": "False", "WheelchairAccessible": "True", "WiFi": "'free'"}
cuisine=>"Tex-Mex"
DogsAllowed=>None
OutdoorSeating=>True
borough=>"Manhattan"
address=>{"building": "627", "coord": [-73.975981, 40.745132], "street": "2 Avenue", "zipcode": "10016"}
_id=>{'$oid': '

In [ ]:
llama_documents[0]

Document(id_='93d3f08d-85f3-494d-a057-19bc834abc29', embedding=None, metadata={'restaurant_id': '40366661', 'attributes': '{"Alcohol": "\'none\'", "Ambience": "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': False}", "BYOB": null, "BestNights": null, "BikeParking": null, "BusinessAcceptsBitcoin": null, "BusinessAcceptsCreditCards": null, "BusinessParking": "None", "Caters": "True", "DriveThru": null, "GoodForDancing": null, "GoodForKids": "True", "GoodForMeal": null, "HasTV": "True", "Music": null, "NoiseLevel": "\'average\'", "RestaurantsAttire": "\'casual\'", "RestaurantsDelivery": "True", "RestaurantsGoodForGroups": "True", "RestaurantsReservations": "True", "RestaurantsTableService": "False", "WheelchairAccessible": "True", "WiFi": "\'free\'"}', 'cuisine': '"Tex-Mex"', 'DogsAllowed': None, 'OutdoorSeating': True, 'borough': '"Manhattan"', 'address': '{"buildin

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(llama_documents)
# 25k nodes takes about 10 minutes, will trim it down to 2.5k
new_nodes = nodes[:2500]

# There are 25k documents, so we need to do batching. Fortunately LlamaIndex provides good batching
# for embedding models, and we are going to rely on the __call__ method for the model to handle this
node_embeddings = embed_model(new_nodes)

In [ ]:
for idx, n in enumerate(new_nodes):
    n.embedding = node_embeddings[idx].embedding
    if "_id" in n.metadata:
        del n.metadata["_id"]

Ensure your database, collection and vector store index is setup on MongoDB Atlas for the collection or the following step won't work appropriately on MongoDB.


 - For assistance with database cluster setup and obtaining the URI, refer to this [guide](https://www.mongodb.com/docs/guides/atlas/cluster/) for setting up a MongoDB cluster, and this [guide](https://www.mongodb.com/docs/guides/atlas/connection-string/) to get your connection string. 

 - Once you have successfully created a cluster, create the database and collection within the MongoDB Atlas cluster by clicking “+ Create Database”. The database will be named movies, and the collection will be named movies_records.

 - Creating a vector search index within the movies_records collection is essential for efficient document retrieval from MongoDB into our development environment. To achieve this, refer to the official [guide](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-index/) on vector search index creation.



In [ ]:
import pymongo


def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


# set up Fireworks.ai Key
import os
import getpass

mongo_uri = getpass.getpass("MONGO_URI:")
if not mongo_uri:
    print("MONGO_URI not set")

mongo_client = get_mongo_client(mongo_uri)

DB_NAME = "whatscooking"
COLLECTION_NAME = "restaurants"

db = mongo_client[DB_NAME]
collection = db[COLLECTION_NAME]

Connection to MongoDB successful


In [ ]:
# To ensure we are working with a fresh collection
# delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff00000000000001ce'), 'opTime': {'ts': Timestamp(1708970193, 3), 't': 462}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708970193, 3), 'signature': {'hash': b'\x9a3H8\xa1\x1b\xb6\xbb\xa9\xc3x\x17\x1c\xeb\xe9\x03\xaa\xf8\xf17', 'keyId': 7294687148333072386}}, 'operationTime': Timestamp(1708970193, 3)}, acknowledged=True)

In [ ]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

vector_store = MongoDBAtlasVectorSearch(
    mongo_client,
    db_name=DB_NAME,
    collection_name=COLLECTION_NAME,
    index_name="vector_index",
)
vector_store.add(new_nodes)

now make sure you create the search index with the right name here

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext

index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
%pip install -q matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pprint
from llama_index.core.response.notebook_utils import display_response

query_engine = index.as_query_engine()

query = "search query: Anything that doesn't have alcohol in it"

response = query_engine.query(query)
display_response(response)
pprint.pprint(response.source_nodes)

**`Final Response:`** Based on the context provided, two restaurant options that don't serve alcohol are:

1. "Academy Restauraunt" in Brooklyn, which serves American cuisine and has a variety of dishes such as Mozzarella sticks, Cheeseburger, Baked potato, Breadsticks, Caesar salad, Chicken parmesan, Pigs in a blanket, Chicken soup, Mac & cheese, Mushroom swiss burger, Spaghetti with meatballs, and Mashed potatoes.

2. "Gabriel'S Bar & Grill" in Manhattan, which specializes in Italian cuisine and offers dishes like Cheese Ravioli, Neapolitan Pizza, assorted gelato, Vegetarian Baked Ziti, Vegetarian Broccoli Pizza, Lasagna, Buca Trio Platter, Spinach Ravioli, Pasta with ricotta cheese, Spaghetti, Fried calamari, and Alfredo Pizza.

Both restaurants offer outdoor seating, are kid-friendly, and have a casual dress code. They also provide take-out service and have happy hour promotions.

[NodeWithScore(node=TextNode(id_='5405e68c-19f2-4a65-95d7-f880fa6a8deb', embedding=None, metadata={'restaurant_id': '40385767', 'attributes': '{"Alcohol": "u\'beer_and_wine\'", "Ambience": "{\'touristy\': False, \'hipster\': False, \'romantic\': False, \'divey\': False, \'intimate\': None, \'trendy\': None, \'upscale\': False, \'classy\': False, \'casual\': True}", "BYOB": null, "BestNights": "{\'monday\': False, \'tuesday\': False, \'friday\': True, \'wednesday\': False, \'thursday\': False, \'sunday\': False, \'saturday\': True}", "BikeParking": "True", "BusinessAcceptsBitcoin": "False", "BusinessAcceptsCreditCards": "True", "BusinessParking": "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", "Caters": "True", "DriveThru": null, "GoodForDancing": "False", "GoodForKids": "True", "GoodForMeal": "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", "HasTV": "True", "Music":